In [16]:
# 10:17pm

import os

import numpy as np
import pandas as pd
from trackml.dataset import load_event, load_dataset, load_event_hits, load_event_cells
from trackml.randomize import shuffle_hits
from trackml.score import score_event
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
#from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.cluster.dbscan_ import dbscan
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
import operator
import time
%matplotlib inline

event_prefix = 'event000000000'
hits = load_event_hits(os.path.join('test/', event_prefix))
cells = load_event_cells(os.path.join('test/', event_prefix))


hit_counts = hits.shape[0]
regr_coef = 0.1165261
regr_intercept =  -1974.7525724298102

particle_count = hit_counts * regr_coef + regr_intercept

particle_count = int(particle_count)
particle_count

11893

In [10]:
hits_playground = hits[["hit_id","x","y","z"]]
hits_playground["r"] =np.sqrt(hits_playground["x"]**2 + hits_playground["y"]**2 + hits_playground["z"]**2)
hits_playground["phi"] = np.degrees(np.arctan2(hits_playground["y"], hits_playground["x"]))
hits_playground["theta"] = np.degrees(np.arccos(hits_playground["z"]/hits_playground["r"]))

hits_playground

/Users/yao-yulin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/yao-yulin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,hit_id,x,y,z,r,phi,theta
0,1,-60.826698,-4.160230,-1502.5,1503.736572,-176.087341,177.676270
1,2,-84.729401,-7.295280,-1502.5,1504.904785,-175.078918,176.760498
2,3,-42.592999,5.048750,-1502.5,1503.112061,173.239990,178.364807
3,4,-62.497501,2.729920,-1502.5,1503.801758,177.498871,177.615829
4,5,-66.203697,-10.601200,-1502.5,1503.995239,-170.902451,177.444870
5,6,-63.056301,-3.073910,-1502.5,1503.825684,-177.209106,177.594009
6,7,-100.077003,-9.506260,-1502.5,1505.859253,-174.573776,176.172226
7,8,-56.604599,-1.169750,-1502.5,1503.566284,-178.816116,177.842056
8,9,-60.591999,2.177720,-1502.5,1503.722900,177.941620,177.689117
9,10,-56.121601,-9.457400,-1502.5,1503.577515,-170.434586,177.830734


In [11]:
X = hits_playground[["theta", "phi"]]
X

In [ ]:
t1 = time.time()

batch_size = 45

mbk = MiniBatchKMeans(n_clusters=particle_count, batch_size=batch_size,
                      n_init=10, max_no_improvement=10, verbose = 0)

mbk.fit(X)

mbk.labels_

t2 = time.time()
print("mbk_time:", t2 - t1)

/Users/yao-yulin/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:1381: RuntimeWarning: init_size=135 should be larger than k=11893. Setting it to 3*k
  init_size=init_size)
